In [39]:
import sys 
sys.path.insert(0,'/home/mohit.kumargupta/deep_boltzmann/')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import keras
import tensorflow as tf

In [40]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

In [41]:
rcParams.update({'font.size': 16})

In [42]:
# Switch AUTORELOAD ON. Disable this when in production mode!
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
from deep_boltzmann.models import ParticleDimer
from deep_boltzmann.networks.invertible import invnet, EnergyInvNet, create_RealNVPNet
from deep_boltzmann.sampling import GaussianPriorMCMC
from deep_boltzmann.networks.plot import test_xz_projection, test_generate_x
from deep_boltzmann.util import count_transitions
from deep_boltzmann.sampling.analysis import free_energy_bootstrap, mean_finite, std_finite
from deep_boltzmann.networks.training import MLTrainer, FlexibleTrainer
from deep_boltzmann.util import save_obj, load_obj

In [6]:
from deep_boltzmann.sampling.analysis import free_energy_bootstrap, mean_finite, std_finite

In [7]:
from openmmtools.integrators import *

In [8]:
from simtk import *
from simtk.openmm import * 
from simtk.unit import *
from simtk.openmm.app import *
from deep_boltzmann.models.openmm import OpenMMEnergy
from deep_boltzmann import openmmutils
import simtk

In [9]:
import mdtraj as md

In [10]:
# reweighting
def test_sample_rew(network, rcfunc, rcmin, rcmax, temperature=1.0, nsample=100000):
    sample_z, sample_x, energy_z, energy_x, log_w = network.sample(temperature=1.0, nsample=nsample)
    bin_means, Es = free_energy_bootstrap(rcfunc(sample_x), rcmin, rcmax, 100, sample=100, weights=np.exp(log_w))
    fig = plt.figure(figsize=(5, 4))
    # model.plot_dimer_energy()
    plt.ylim(-10, 20)
    Emean = mean_finite(Es, axis=0)-7
    Estd = std_finite(Es, axis=0)
    plt.errorbar(bin_means, Emean, 2*Estd)
    # variance
    var = mean_finite(std_finite(Es, axis=0) ** 2)
    print('Estimator Standard Error: ', np.sqrt(var))
    return fig, bin_means, Emean, Estd

In [11]:
def latent_interpolation(bg, x1, x2, nstep=1000, through_origin=False):
    lambdas = np.array([np.linspace(0, 1, num=nstep)]).T
    x1 = np.array([x1])
    x2 = np.array([x2])
    z1 = bg.transform_xz(x1)
    z2 = bg.transform_xz(x2)
    if through_origin:
        zpath1 = z1 * (1-lambdas[::2])
        zpath2 = z2 * (lambdas[::2]) 
        zpath = np.vstack([zpath1, zpath2])
    else:
        zpath = z1 + lambdas*(z2-z1)
    xpath = bg.transform_zx(zpath)
    return xpath

In [12]:
def low_energy_fraction(energies, Emax):
    low_energy_count = [np.count_nonzero(E<Emax) for E in energies]
    sizes = [E.size for E in energies]
    low_energy_fraction = np.array(low_energy_count) / sizes
    return low_energy_fraction

In [13]:
def plot_convergence(hist_ML, hist_KL, enerx_cut, enerz_cut, MLcol=1, KLcol=2):
    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(5, 10))
    niter1 = len(hist_ML[0])
    niter2 = hist_KL[1].shape[0]
    niter = niter1 + niter2
    # ML loss
    losses_ML = np.concatenate([hist_ML[0], hist_KL[1][:, MLcol]])
    xticks = np.arange(niter1 + niter2) + 1
    axes[0].plot(xticks, losses_ML, color='black')
    axes[0].set_xlim(0, niter + 1)
    axes[0].set_ylabel('ML loss')
    axes[0].axvline(x=200, color='red', linestyle='--', linewidth=3)
    # KL loss
    losses_KL = hist_KL[1][:, KLcol]
    xticks = np.arange(niter1, niter1 + niter2) + 1
    axes[1].plot(xticks, losses_KL, color='black')
    axes[1].set_xlim(0, niter + 1)
    axes[1].set_ylabel('KL loss')
    axes[1].axvline(x=200, color='red', linestyle='--', linewidth=3)
    # low energy fractions
    enerx = hist_ML[2] + hist_KL[3]
    enerz = hist_ML[3] + hist_KL[4]
    lef_x = low_energy_fraction(enerx, enerx_cut)
    lef_z = low_energy_fraction(enerz, enerz_cut)
    axes[2].plot(lef_x, color='black', label='x')
    axes[2].plot(lef_z, color='blue', label='z')
    axes[2].set_xlim(0, niter + 1)
    axes[2].set_ylim(0, 1.05)
    axes[2].axvline(x=200, color='red', linestyle='--', linewidth=3)
    axes[2].set_ylabel('Training iterations')
    axes[2].set_ylabel('Low energy fraction')
    axes[2].legend()
    return fig, axes

In [14]:
top = '/home/mohit.kumargupta/system.pdb'
filename = '/home/mohit.kumargupta/FoldedTRPCage.xtc'

In [15]:
traj = md.load(filename,top = top)
import ast
#params = ast.literal_eval(str(trajdict['params']))
x = traj[:].xyz


In [ ]:
print(x)

In [16]:
topology = traj.topology

In [21]:
print(topology.select('backbone'))

[  0   1   2   3  14  15  16  17  24  25  26  27  45  46  47  48  55  56
  57  58  72  73  74  75  96  97  98  99 115 116 117 118 125 126 127 128
 137 138 139 140 144 145 146 147 151 152 153 154 165 166 167 168 176 177
 178 179 187 188 189 190 194 195 196 197 218 219 220 221 232 233 234 235
 246 247 248 249 260 261 262 263]


In [17]:
index = topology.select('backbone')


In [18]:
IC_Index = []
for i in range(272):
    if(i not in index):
        IC_Index.append(i)

print(IC_Index)

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 18, 19, 20, 21, 22, 23, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 49, 50, 51, 52, 53, 54, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 119, 120, 121, 122, 123, 124, 129, 130, 131, 132, 133, 134, 135, 136, 141, 142, 143, 148, 149, 150, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 169, 170, 171, 172, 173, 174, 175, 180, 181, 182, 183, 184, 185, 186, 191, 192, 193, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 264, 265, 266, 267, 268, 269, 270, 271]


In [21]:
batchsize_ML =  256
batchsize_KL = 1000
modeller = Modeller(PDBObject.topology, PDBObject.positions)

In [22]:
integrator = LangevinIntegrator(300* kelvin,
                   1.0/ picosecond,
                    0.002*picoseconds)

In [23]:
forcefield = ForceField('amber99sb.xml', 'tip3p.xml')
# setup BPTI
INTEGRATOR_ARGS = (300* kelvin,
                   1.0/ picoseconds,
                    0.002*picoseconds)

In [24]:
PDBObject = PDBFile(top)

In [25]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod= CutoffNonPeriodic, nonbondedCutoff=1.0*nanometers, constraints= AllBonds)

In [26]:
from deep_boltzmann.models.proteins import mdtraj2Z
Z_ = np.array(mdtraj2Z(topology))

In [27]:
print(modeller.topology)

<Topology; 1 chains, 20 residues, 272 atoms, 278 bonds>


In [28]:
print (Z_)

[[ 11   0   1   2]
 [  3   2   1   0]
 [  8   1   0   2]
 [  4   1   0   2]
 [  5   4   1   0]
 [  6   5   4   1]
 [  7   5   4   6]
 [  9   4   1   5]
 [ 10   4   1   5]
 [ 12   0   1  11]
 [ 13   0   1  12]
 [ 19  14  15  16]
 [ 17  16  15  14]
 [ 20  15  14  16]
 [ 18  15  14  16]
 [ 21  18  15  14]
 [ 22  18  15  21]
 [ 23  18  15  22]
 [ 36  24  25  26]
 [ 27  26  25  24]
 [ 37  25  24  26]
 [ 28  25  24  26]
 [ 29  28  25  24]
 [ 30  29  28  25]
 [ 31  29  28  30]
 [ 32  30  29  28]
 [ 33  31  29  30]
 [ 34  32  33  30]
 [ 35  34  32  33]
 [ 38  28  25  29]
 [ 39  28  25  29]
 [ 40  30  29  32]
 [ 41  31  29  33]
 [ 42  32  30  34]
 [ 43  33  31  34]
 [ 44  35  34  32]
 [ 50  45  46  47]
 [ 48  47  46  45]
 [ 51  46  45  47]
 [ 49  46  45  47]
 [ 52  49  46  45]
 [ 53  49  46  52]
 [ 54  49  46  53]
 [ 64  55  56  57]
 [ 58  57  56  55]
 [ 65  56  55  57]
 [ 59  56  55  57]
 [ 60  59  56  55]
 [ 61  60  59  56]
 [ 63  61  60  59]
 [ 62  61  60  63]
 [ 66  59  56  60]
 [ 67  59  5

In [29]:
EnergyModel = OpenMMEnergy(system, LangevinIntegrator , nanometers, n_atoms= 272, openmm_integrator_args=INTEGRATOR_ARGS )

In [30]:
print(index[5])

15


In [31]:
print(np.array(new)[0,] )

NameError: name 'new' is not defined

In [ ]:
data = np.array([[11, 22], [33, 44], [55, 66]])
# index data
print(data[0,])

In [32]:
new=[]
for snapshot in x:
    new.append(snapshot.flatten('F'))

In [33]:
print(len(new[0]))

816


In [44]:
bg = invnet(3*272, 'RRRRRRRR', energy_model=EnergyModel, nl_layers=4, nl_hidden=100, #100
            nl_activation='relu', nl_activation_scale='tanh', whiten=None, ic_cart = index, ic = Z_, ic_norm = np.array(new) )

I<RRRRRRRR>
I 816 0 0


/home/mohit.kumargupta/deep_boltzmann/deep_boltzmann/models/MM.py:20: RuntimeWarning: invalid value encountered in arccos
  angle = np.degrees(np.arccos(cosine_angle))


< 810 0 6
R 405 405 6
R 405 405 6
R 405 405 6
R 405 405 6
R 405 405 6
R 405 405 6
R 405 405 6
R 405 405 6
> 405 405 6


In [45]:
hist_bg_ML = bg.train_ML(np.array(new), xval = np.array(new) , epochs=200, lr=0.00001, batch_size=batchsize_ML, 

                         std=1.0, verbose=1, return_test_energies=True)

InternalError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version